In [1]:
#!/usr/bin/env python3

"""
.. module:: rt_assignment2
   :platform: Unix
   :synopsis: Python module for the node_a
.. moduleauthor:: Mohsen Kashefi

This code implements the node_a.

Subscriber:
    - /node_a/pose: Subscribes to the pose.
"""

import rospy
from geometry_msgs.msg import Point, Pose, Twist
from nav_msgs.msg import Odometry
import actionlib
from assignment_2_2023.msg import Vel
from assignment_2_2023.msg import PlanningAction, PlanningGoal
from std_srvs.srv import SetBool
from actionlib_msgs.msg import GoalStatus
import ipywidgets as widgets
import jupyros as jr
import matplotlib.pyplot as plt
import numpy as np
import tf
from tf.transformations import quaternion_matrix
from matplotlib.animation import FuncAnimation



In [2]:
ris_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>ROS Node Explanation</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            line-height: 1.6;
            margin: 20px;
        }
        h1, h2, h3 {
            color: #333;
        }
        code {
            background-color: #f4f4f4;
            padding: 2px 4px;
            border-radius: 4px;
        }
        pre {
            background-color: #f4f4f4;
            padding: 10px;
            border-radius: 4px;
        }
    </style>
</head>
<body>
    <h1>ROS Node Explanation</h1>
    <p>This script is designed for a ROS (Robot Operating System) environment. It facilitates interaction with a robot's navigation goals using Jupyter widgets. It allows users to input new goals or cancel existing ones and visualizes the robot's path and goal statuses.</p>
    
    <h2>Script Overview</h2>
    
    <h3>Imports and Global Variables</h3>
    <p>The script imports necessary ROS packages and other libraries, such as <code>rospy</code>, <code>geometry_msgs</code>, <code>nav_msgs</code>, and others for messaging and plotting. It initializes several global variables to store data points, publishers, and counters.</p>
    
    <h3>Callback Functions</h3>
    <p><strong>odom_callback:</strong> This function receives odometry messages and updates the position data, which is used for plotting the robot's path.</p>
    <p><strong>publish_position_and_velocity:</strong> Publishes the robot's current position and velocity.</p>
    
    <h3>Helper Functions</h3>
    <p><strong>get_new_goal:</strong> Retrieves a new goal based on user input from Jupyter widgets and sets ROS parameters.</p>
    <p><strong>update_chart:</strong> Updates a chart displaying the count of reached and cancelled goals.</p>
    
    <h3>Main Functionality</h3>
    <p><strong>handle_goal_commands:</strong> Initializes ROS node, sets up publishers, subscribers, and action clients. Handles user commands for setting or cancelling goals.</p>
    <p><strong>on_button_clicked:</strong> Handles button click events to process commands and goals.</p>
    
    <h3>Widgets and Visualization</h3>
    <p>The script uses Jupyter widgets (<code>ipywidgets</code>) to create an interactive interface for user inputs. It also uses Matplotlib for real-time plotting of the robot's trajectory.</p>
    
    <h3>Execution Block</h3>
    <p>The <code>if __name__ == '__main__':</code> block initializes widgets, sets up the visualization, and calls the main handler function.</p>
    
    <h2>Interactive Widgets and Visualization</h2>
    
    <p>The script provides an interactive interface for users via Jupyter Notebook widgets:</p>
    <ul>
        <li><strong>Input Widgets:</strong>
            <ul>
                <li><code>input_x_widget</code>: Text input for the x-coordinate of the new goal.</li>
                <li><code>input_y_widget</code>: Text input for the y-coordinate of the new goal.</li>
                <li><code>input_widget</code>: Dropdown menu for command selection (new goal or cancel goal).</li>
            </ul>
        </li>
        <li><strong>Button:</strong> A button to submit the command and goal inputs.</li>
        <li><strong>Output:</strong> Displays the widgets and updates based on user interaction.</li>
        <li><strong>Plot:</strong> A real-time plot using Matplotlib to visualize the robot's path.</li>
    </ul>
    
    <h3>Detailed Breakdown</h3>
    
    <h4>Setting Up ROS Node and Publishers</h4>
    <p>Initializes a ROS node and sets up publishers for position and velocity messages.</p>
    
    <h4>Action Client and Subscriber</h4>
    <p>Creates an action client to communicate with the goal-reaching action server and subscribes to the <code>/odom</code> topic to receive odometry messages for real-time updates.</p>
    
    <h4>Handling User Commands</h4>
    <p>Depending on the user's command (<code>d</code> for a new goal, <code>q</code> for cancel goal), it sends a new goal to the action server or cancels the current goal. Updates the goal counts and refreshes the chart.</p>
    
    <h4>Visualization</h4>
    <p>Real-time plotting of the robot's path based on odometry data and a bar chart to show the count of reached and cancelled goals.</p>
    
    <h4>Exception Handling</h4>
    <p>The script includes basic exception handling for invalid inputs.</p>
    
    <h3>Conclusion</h3>
    <p>This script is designed to run within a Jupyter Notebook environment, enabling users to interactively set and manage robot goals while visualizing the robot's movements and goal statuses in real-time. It leverages ROS for robotic communication and control, Matplotlib for plotting, and <code>ipywidgets</code> for the interactive interface.</p>
</body>
</html>

"""

html_widget = widgets.HTML(value=ris_html)
display(html_widget)

HTML(value='\n<!DOCTYPE html>\n<html lang="en">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport"…

In [3]:
x_data = []
y_data = []
# arrow = None
%matplotlib widget
# fig, ax = plt.subplots()
# ax.set_xlim((-20, 20))
# ax.set_ylim((-20, 20))
# ax.set_title("my_robot")
# ax.set_xlabel("x")
# ax.set_ylabel("y")
# line, = ax.plot([], [], 'ro')

pos_vel_publisher = None  # Define pos_vel_publisher in global scope
goal_cancelled = False
goal_reached_count = 0
goal_cancelled_count =0

In [4]:
import matplotlib.pyplot as plt
import rospy
import tf
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
%matplotlib widget
class Visualiser:
	def __init__(self):
		self.fig, self.ax = plt.subplots()
		self.ln, = plt.plot([], [], 'ro')
		self.x_data, self.y_data = [] , []
#         self.ax.set_title("my_robot")
#         self.ax.set_xlabel("x")
#         self.ax.set_ylabel("y")

	def plot_init(self):
		self.ax.set_xlim(-20, 20)
		self.ax.set_ylim(-20, 20)
		return self.ln,
		
	def odom_callback(self, msg):
		self.y_data.append(msg.pose.pose.position.y)
		self.x_data.append(msg.pose.pose.position.x)

	def update_plot(self, frame):
		self.ln.set_data(self.x_data, self.y_data)
		return self.ln,
    
def goal_done_cb(state, result):
    global goal_reached_count, goal_cancelled_count
    with output:
        if state == actionlib.GoalStatus.SUCCEEDED:
            goal_reached_count += 1
#             goal_status.value = "Goal reached"
        else:
            goal_cancelled_count += 1
#             goal_status.value = "Goal failed/canceled"
        update_chart()

# Function to handle goal feedback
def goal_feedback_cb(feedback):
    with output:
        pass
    
def get_new_goal():
    """
    Retrieves a new goal from user input.

    :return: New goal and its coordinates
    :rtype: tuple(assignment_2_2023.msg.PlanningGoal, tuple(float, float))
    """
    try:
        input_x = float(input_x_widget.value)
        input_y = float(input_y_widget.value)
    except ValueError:
        rospy.logwarn("Invalid input. Please enter a valid number.")
        return None, None

    rospy.set_param('/des_pos_x', input_x)
    rospy.set_param('/des_pos_y', input_y)

    goal = PlanningGoal()
    goal.target_pose.pose.position.x = input_x
    goal.target_pose.pose.position.y = input_y

    return goal, (input_x, input_y)


def odom_callback(msg):
    global x_data, y_data
    y_data.append(msg.pose.pose.position.y)
    x_data.append(msg.pose.pose.position.x)
    if pos_vel_publisher:
        publish_position_and_velocity(msg)
#     line.set_xdata(x_data)
#     line.set_ydata(y_data)
#     ax.relim()
#     ax.autoscale_view()


def publish_position_and_velocity(msg):
    """
    Publishes the current position and velocity information.

    :param msg: Odometry message containing position and velocity information.
    :type msg: nav_msgs.msg.Odometry
    """
    current_pos = msg.pose.pose.position
    current_vel_linear = msg.twist.twist.linear
    current_vel_angular = msg.twist.twist.angular

    pos_and_vel = Vel()
    pos_and_vel.pos_x = current_pos.x
    pos_and_vel.pos_y = current_pos.y
    pos_and_vel.vel_x = current_vel_linear.x
    pos_and_vel.vel_z = current_vel_angular.z

    pos_vel_publisher.publish(pos_and_vel)
    
def update_chart():
    # Update the chart with the latest counts
    labels = ['Reached Goals', 'Cancelled Goals']
    counts = [goal_reached_count , goal_cancelled_count]

    ax.clear()
    ax.bar(labels, counts, color=['green', 'red'])
    ax.set_ylim(0, max(counts) + 1)
    plt.show()

# def on_button_clicked2(b):

#     fig, ax = plt.subplots()
#     ax.set_xlim((-20, 20))
#     ax.set_ylim((-20, 20))
#     ax.set_title("my_robot")
#     ax.set_xlabel("x")
#     ax.set_ylabel("y")
#     line, = ax.plot(x_data, y_data, 'ro')


def handle_goal_commands(button):
    global pos_vel_publisher  # Use the global pos_vel_publisher variable
   
    rospy.init_node('set_target_client') 
    pos_vel_publisher = rospy.Publisher("/pos_vel", Vel, queue_size=1)
    action_client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
    action_client.wait_for_server()
#     vis = Visualiser()
    rospy.Subscriber("/odom",Odometry , odom_callback)
#     rospy.Subscriber("/odom",Odometry ,odom_callback)
#     ani = FuncAnimation(fig, update, init_func=plot_init)
#     plt.show(block=True)
#     sub = rospy.Subscriber('/pose_v', Pose, vis.odom_callback)
#     ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
#     plt.show(block=True)

    def on_button_clicked(b ):
        global goal_reached_count, goal_cancelled_count
        target_pos_x = rospy.get_param('/des_pos_x')
        target_pos_y = rospy.get_param('/des_pos_y')

        current_goal = PlanningGoal()
        current_goal.target_pose.pose.position.x = target_pos_x
        current_goal.target_pose.pose.position.y = target_pos_y
        rospy.loginfo("Current goal: target_x = %f, target_y = %f", target_pos_x, target_pos_y)

        command = input_widget.value
        goal_cancelled = False
        if command == 'd':
            new_goal, new_goal_coords = get_new_goal()
            if new_goal:
                print(new_goal)
                action_client.send_goal(new_goal, done_cb=goal_done_cb, feedback_cb=goal_feedback_cb)
                goal_cancelled = False
#                 goal_reached_count += 1  # Increment reached goals count
        elif command == 'q':
            if not goal_cancelled:
                goal_cancelled = True
                action_client.cancel_goal()
                rospy.loginfo("Current goal has been cancelled")
#                 goal_cancelled_count += 1  # Increment cancelled goals count
            else:
                rospy.loginfo("No active goal to cancel")
        else:
            rospy.logwarn("This is an incorrect command. Please choose between 'd' and 'q'.")

        rospy.loginfo("Last received goal: target_x = %f, target_y = %f", current_goal.target_pose.pose.position.x,
                      current_goal.target_pose.pose.position.y)
        rospy.sleep(1)

    button.on_click(on_button_clicked)
    
#     button2.on_click(on_button_clicked2)

In [5]:
if __name__ == '__main__':
    
    input_x_widget = widgets.Text(description='x:')
    input_y_widget = widgets.Text(description='y:')
#     input_widget = widgets.Text(description='Command:')
    input_widget = widgets.Dropdown(
        options=[('new goal', 'd'), ('cancel goal', 'q')],
        value='d',
        description='Command:',
    )
    button = widgets.Button(description="Submit")

#     jr.subscribe('/odom', Odometry, odom_callback)

    output = widgets.Output()
    display(input_x_widget, input_y_widget, input_widget, button, output)
    
#     rospy.init_node('odom_visualizer_node')
    
#     vis = Visualiser()
#     sub = rospy.Subscriber('/pose_v', Pose, vis.odom_callback)
#     ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
#     plt.show(block=True)

    fig, ax = plt.subplots()
     
    handle_goal_commands(button)
    

#     plt.show()

Text(value='', description='x:')

Text(value='', description='y:')

Dropdown(description='Command:', options=(('new goal', 'd'), ('cancel goal', 'q')), value='d')

Button(description='Submit', style=ButtonStyle())

Output()

[INFO] [1716871985.338313, 1589.654000]: Current goal: target_x = 4.000000, target_y = 4.000000
target_pose: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: ''
  pose: 
    position: 
      x: 2.0
      y: 2.0
      z: 0.0
    orientation: 
      x: 0.0
      y: 0.0
      z: 0.0
      w: 0.0
[INFO] [1716871985.362894, 1589.666000]: Last received goal: target_x = 4.000000, target_y = 4.000000


In [6]:

vis = Visualiser()
sub = rospy.Subscriber('/odom', Odometry, vis.odom_callback)
ani = FuncAnimation(vis.fig, vis.update_plot, init_func=vis.plot_init)
plt.show(block=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …